In [1]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [10]:
new = pd.read_csv("part2.csv", sep='\t') #or pd.read_csv("part3.csv", sep='\t') for third part
new = pd.DataFrame(new.iloc[:,1])
new.columns = ['location']
new['canton'] = 'none'
new.head()

,location,canton
0,"47.368166, 8.523833",none
1,"46.849809, 9.532291",none
2,"46.84986, 9.532248",none
3,"46.849809, 9.532313",none
4,"46.849794, 9.532281",none


In [11]:
exc = [] #to store exception batches
l = [] #to store timeout location

In [12]:
from time import sleep

def addCanton(row):
    loc = row
    try:
        sleep(0.01) # Time in seconds.
        geolocator = Nominatim()
        location = geolocator.reverse(loc)
        try:
            canton = location.raw["address"]["state"]
            return canton
        except KeyError:
            return None
    except GeocoderTimedOut:
        print("Exception: " + str(row))
        exc.append(loc)
        return 'none'

In [13]:
new.shape[0]

322108

### Whenever there are problems with batches, you can stop the cycle (all the batches that have finished are stored) and change the range values to start over from where you need. Also, there is 108 values left that would need to be done after the whole cycle is done

In [ ]:
for i in range(0, 322000, 500):
    print("starting" + str(i))
    try:
        new['canton'][i:(i+500)] = new.iloc[i:(i+500), 0].apply(addCanton)
    except GeocoderServiceError:
        print("Exception with batch starting at "+str(i))
        l.append(i)
        sleep(600)
    sleep(300)

In [ ]:
#to be done when the previous cycle finished completely
new['canton'][322000:322108] = new.iloc[322000:322108, 0].apply(addCanton)
#in case of GeocoderServiceError, which is usually caused by too many requests, wait some time and then try again

### Now it's important to save the dataframe, so it can be used later without redoing the whole process again. Also, check if this dataframe contains any 'none' field in canton column. If there is (could be because of timeout), they need to be done again, with the help of the same line above (but with the right indices).

In [ ]:
import pickle

# this is to save to pickle
with open('canton_loc.pickle', 'wb') as handle:
    pickle.dump(new, handle, protocol=pickle.HIGHEST_PROTOCOL)

# this is to load from pickled data
with open('canton_loc.pickle', 'rb') as handle:
    new = pickle.load(handle)